# Quantity

## Getting Started

Before we start, let's import the symbols we're going to use from `fv3gfs.util`, as well as `numpy` and `gt4py` to allocate data.

In [3]:
from fv3gfs.util import Quantity, X_DIM, Y_DIM, Z_DIM, X_INTERFACE_DIM, Y_INTERFACE_DIM, Z_INTERFACE_DIM
import numpy as np
import gt4py

## A Quick Look

Let's check out some of the features we just went over in slides. We're going to initialize a quantity a couple different ways, check out how the `quantity.storage` and `quantity.data` attributes relate to each other, and do a couple indexing operations.

### Initialization

First, we'll show how you can create a Quantity using a numpy array. The initialization routine has three required arguments (data, dims, and units), and three optional arguments (origin, extent, and gt4py_backend).

In [4]:
help(Quantity.__init__)

Help on function __init__ in module fv3gfs.util.quantity:

__init__(self, data, dims: Sequence[str], units: str, origin: Sequence[int] = None, extent: Sequence[int] = None, gt4py_backend: Union[str, NoneType] = None)
    Initialize a Quantity.
    
    Args:
        data: ndarray-like object containing the underlying data
        dims: dimension names for each axis
        units: units of the quantity
        origin: first point in data within the computational domain
        extent: number of points along each axis within the computational domain
        gt4py_backend: backend to use for gt4py storages, if not given this will
            be derived from a Storage if given as the data argument, otherwise the
            storage attribute is disabled and will raise an exception



This cell shows that you can create a quantity without specifying `gt4py_backend`, but if you do you won't be able to access its `.storage` attribute. In this workshop, be sure to supply a backend.

In [5]:
array = np.zeros([6, 6])
quantity = Quantity(
    array,
    origin=(1, 1),
    extent=(4, 4),
    dims=[X_DIM, Y_DIM],
    units="degK"
    # we're not passing in gt4py_backend
)

print(quantity.storage)  # this line will trigger a TypeError

TypeError: Quantity was initialized with a non-storage type and no gt4py backend was given

You will also notice we imported some symbols for dimension types. When you initialize a quantity, you must use those symbols in the `dims` if you plan to use other dimensionally-aware features of `fv3gfs.util`. For example, the halo updates use those particular values to determine the orientation of the array when copying halo regions.

To access a storage, you need to give a `gt4py_backend` when you initialize the Quantity. For CPU, you likely want to use "numpy", "gtx86", or "gtmc". For GPU, you want to use "gtcuda".

Also a quick note that `units` is a required input. Technically you can insert an empty string `""` for units and the code wil run. If you're going to do this instead of inserting unit information, we suggest at least using `"unknown"` or a similar string, since an empty string is a unit (unitless).

Keep in mind when deciding whether to write units that for less time than you need to take a sip of your morning coffee, you could save a grad student (or your future self) weeks of debugging their code.

### Indexing and view

Now that we can create a Quantity, we can use it for indexing. You should be able to write indexing code which is aware of the locations of the compute domain edges without writing down those locations, because the Quantity already knows them.

A very common operation is to retrieve the entire compute domain with no halos. Quantity provides a convenient way to interact with and index on the compute domain for an array which contains halos.

In [8]:
quantity.view[:] = np.arange(16).reshape([4, 4])
print(quantity.view[:])
print(type(quantity.view))
print(type(quantity.view[:]))

[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]
<class 'fv3gfs.util.quantity.BoundedArrayView'>
<class 'numpy.ndarray'>


Note that `view` itself is not a numpy array. It is a custom object which provides indexing operations, and indexing returns a numpy array.

If we look at the entire array, we can see the halos are left untouched.

In [11]:
print(quantity.data)

[[ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  2.  3.  0.]
 [ 0.  4.  5.  6.  7.  0.]
 [ 0.  8.  9. 10. 11.  0.]
 [ 0. 12. 13. 14. 15.  0.]
 [ 0.  0.  0.  0.  0.  0.]]


You can also modify the compute domain by modifying an index or slice of view. In the cell below, set the compute domain values to 1.

In [7]:
quantity.data[:] = 0.

# Exercise: set the compute domain values to 1.


print(quantity.data)
# expected:
#[[0. 0. 0. 0. 0. 0.]
# [0. 1. 1. 1. 1. 0.]
# [0. 1. 1. 1. 1. 0.]
# [0. 1. 1. 1. 1. 0.]
# [0. 1. 1. 1. 1. 0.]
# [0. 0. 0. 0. 0. 0.]]

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


Providing indices in `view` has fairly complex behavior, to deal with the complexities of having halos. If you need to get more "normal" indexing, you can first create a numpy array with `view[:]`, and then provide indexing on the "normal" numpy array. For example, -1 does not behave the same as for a numpy array or list, because you may actually want to index into the halo instead of wrapping around to the other edge of the compute domain:

In [9]:
# gives the second through second-last points on each axis
print(quantity.view[:][1:-1, 1:-1])

[[0. 0.]
 [0. 0.]]


In [10]:
# -1 is treated as the halo point before the origin
# since end is before start, gives an empty view
print(quantity.view[1:-1, 1:-1])  

[]


The exact behavior of `quantity.view` and its attributes (explored further in "moving forward") is evolving as we use it more and collect feedback on its use cases. If you have feedback or use cases you'd like to bring to our attention, please contact us about it!

In [11]:
print(help(quantity.view))

Help on BoundedArrayView in module fv3gfs.util.quantity object:

class BoundedArrayView(builtins.object)
 |  BoundedArrayView(array, dims, origin, extent)
 |  
 |  A container of objects which provide indexing relative to corners and edges
 |  of the computational domain for convenience.
 |  
 |  Default start and end indices for all dimensions are modified to be the
 |  start and end of the compute domain. When using edge and corner attributes, it is
 |  recommended to explicitly write start and end offsets to avoid confusion.
 |  
 |  Indexing on the object itself (view[:]) is offset by the origin, and default
 |  start and end indices are modified to be the start and end of the compute domain.
 |  
 |  For corner attributes e.g. `northwest`, modified indexing is done for the two
 |  axes according to the edges which make up the corner. In other words, indexing
 |  is offset relative to the intersection of the two edges which make the corner.
 |  
 |  For `interior`, start indices of

### Storage integration

Quantity has integration both to initialize a Quantity from a GT4py storage object, and to retrieve a storage from the Quantity. The ndarray attribute `quantity.data` shares the same memory as the storage attribute `quantity.storage`, so that modifying one will modify both without performing a copy.

This first cell shows how a Quantity can be used to initialize a storage.

In [12]:
array = np.zeros([6, 6])
quantity = Quantity(
    array,
    origin=(1, 1),
    extent=(4, 4),
    dims=[X_DIM, Y_DIM],
    units="degK",
    gt4py_backend="numpy",
)
print(quantity.storage)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


The `data` attribute and view slices provide a numpy (or if using a GPU storage, cupy) array for the underlying data. Modifying any of these will modify all three.

In [13]:
print(type(quantity.storage))
print(type(quantity.data))

<class 'gt4py.storage.storage.CPUStorage'>
<class 'numpy.ndarray'>


In [14]:
quantity.storage[0, 0] = 1
quantity.storage[1, 1] = 2
quantity.storage[4, 4] = 3
print(quantity.data)
print(quantity.view[:])

[[1. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 3. 0.]
 [0. 0. 0. 0. 0. 0.]]
[[2. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]]


## Moving Forward

Let's go into some more detail about `quantity.view` and its attributes. When you use these view objects, you should think of the start and end indices as being offset from some reference index. This means that a negative index refers to a location before the reference, and not necessarily to the end of the array.

For one attribute (`quantity.view.interior`), the reference is different for the start and end indices.

The behavior of these views may seem non-intuitive if you are familiar with numpy array indexing. For simple analysis code running in the compute domain, you will not need these view attributes. Keep in mind that the use case for these views is to access either halo or compute data near corners and edges of the compute domain, and that the two "weird" behaviors (negatives and the behavior of 0:0 for `quantity.view.interior`) are designed for that use case.

Let's start by creating a quantity with increasing values, so we can tell what regions we're viewing. We'll also use a quantity with two halo points.

In [13]:
# set quantity.data to sequential values so we can see what region we're viewing
array = np.zeros([7, 7])
quantity = Quantity(
    np.arange(49).reshape([7, 7]),
    origin=(2, 2),
    extent=(3, 3),
    dims=[X_DIM, Y_DIM],
    units="degK",
    gt4py_backend="numpy",
)
print(quantity.storage)

[[ 0  1  2  3  4  5  6]
 [ 7  8  9 10 11 12 13]
 [14 15 16 17 18 19 20]
 [21 22 23 24 25 26 27]
 [28 29 30 31 32 33 34]
 [35 36 37 38 39 40 41]
 [42 43 44 45 46 47 48]]


In [15]:
quantity.view.southwest[:]

TypeError: object of type 'slice' has no len()

The closest thing to what we've already seen is `quantity.view.southwest`. This one is actually identical to `quantity.view`, and is provided for completeness.

In [14]:
print(quantity.view.southwest[:2, :2])
print(quantity.view[:2, :2])

[[16 17]
 [23 24]]
[[16 17]
 [23 24]]


Other corners are useful for accessing data in the neighborhood of those corners. For example, this will access the compute data immediately next to the northeast corner:

In [86]:
quantity.view.northeast[-2:, -2:]

array([[21, 22],
       [27, 28]])

while the following will access the halo data next to the northeast corner:

In [95]:
# 0 and 2 are both offsets from the northeast corner
quantity.view.northeast[0:2, 0:2]

array([[40, 41],
       [47, 48]])

Exercise: use `quantity.view.southeast` to print the halo points on the east edge.

In [96]:
print(quantity.view.southeast[???])

SyntaxError: invalid syntax (<ipython-input-96-4cbd891a6068>, line 1)

`quantity.view.interior` may appear awkward, when you see what it returns for this slice:

In [97]:
print(quantity.view.interior[0:0, 0:0])

[[16 17 18]
 [23 24 25]
 [30 31 32]]


The use case for `quantity.view.interior` is to make small modifications on the boundaries that mark the compute domain (called "interior" because compute is an overloaded term).

Exercise: Below, use `quantity.view.interior` to print the quantity including the first ring of halo points but excluding the second ring. The output should look identical to the `data` operation below:

In [101]:
print(quantity.view.interior[???])

SyntaxError: invalid syntax (<ipython-input-101-53879e157e89>, line 1)

In [102]:
print(quantity.data[
    quantity.origin[0] - 1:quantity.origin[0] + quantity.extent[0] + 1,
    quantity.origin[1] - 1:quantity.origin[1] + quantity.extent[1] + 1
])

[[ 8  9 10 11 12]
 [15 16 17 18 19]
 [22 23 24 25 26]
 [29 30 31 32 33]
 [36 37 38 39 40]]


You can see from this example that the amount of boilerplate needed to use `quantity.data` is pretty high, if you don't want your code to break when the size of the domain or halo changes. Once you know what `quantity.view.interior` is doing, it's much easier to read and write than `quantity.data`.

The help routine can give useful information about the attributes and methods available on an object. We encourage you to check out the documentation if you need to remember the name or purpose of a method or attribute.

In [36]:
help(Quantity)

Help on class Quantity in module fv3gfs.util.quantity:

class Quantity(builtins.object)
 |  Quantity(data, dims: Sequence[str], units: str, origin: Sequence[int] = None, extent: Sequence[int] = None, gt4py_backend: Union[str, NoneType] = None)
 |  
 |  Data container for physical quantities.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, data, dims: Sequence[str], units: str, origin: Sequence[int] = None, extent: Sequence[int] = None, gt4py_backend: Union[str, NoneType] = None)
 |      Initialize a Quantity.
 |      
 |      Args:
 |          data: ndarray-like object containing the underlying data
 |          dims: dimension names for each axis
 |          units: units of the quantity
 |          origin: first point in data within the computational domain
 |          extent: number of points along each axis within the computational domain
 |          gt4py_backend: backend to use for gt4py storages, if not given this will
 |              be derived from a Storage if given as t

In [38]:
help(quantity.view)

Help on BoundedArrayView in module fv3gfs.util.quantity object:

class BoundedArrayView(builtins.object)
 |  BoundedArrayView(array, dims, origin, extent)
 |  
 |  A container of objects which provide indexing relative to corners and edges
 |  of the computational domain for convenience.
 |  
 |  Default start and end indices for all dimensions are modified to be the
 |  start and end of the compute domain. When using edge and corner attributes, it is
 |  recommended to explicitly write start and end offsets to avoid confusion.
 |  
 |  Indexing on the object itself (view[:]) is offset by the origin, and default
 |  start and end indices are modified to be the start and end of the compute domain.
 |  
 |  For corner attributes e.g. `northwest`, modified indexing is done for the two
 |  axes according to the edges which make up the corner. In other words, indexing
 |  is offset relative to the intersection of the two edges which make the corner.
 |  
 |  For `interior`, start indices of